<a href="https://colab.research.google.com/github/Gr3gP/Transformer-Builds-/blob/main/HuggingFace_ViT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers
!pip install datasets

     |████████████████████████████████| 2.5MB 4.2MB/s 
     |████████████████████████████████| 3.3MB 23.5MB/s 
     |████████████████████████████████| 901kB 49.9MB/s 
     |████████████████████████████████| 245kB 4.0MB/s 
     |████████████████████████████████| 245kB 28.8MB/s 
     |████████████████████████████████| 122kB 21.9MB/s 


In [30]:
import torch.nn as nn
import numpy as np
import transformers
import datasets

from transformers import ViTFeatureExtractor, ViTModel, ViTForImageClassification, TrainingArguments, Trainer, default_data_collator, EarlyStoppingCallback
from transformers import ViTModel
from transformers.modeling_outputs import  SequenceClassifierOutput
from datasets import load_dataset, load_metric, Features, ClassLabel, Array3D

In [3]:
from datasets import load_dataset

train_ds, test_ds = load_dataset('cifar10', split= ['train[:5000]', 'test[:2000]'])
splits = train_ds.train_test_split(test_size=0.1)
train_ds = splits['train']
val_ds = splits['test']

Dataset cifar10 downloaded and prepared to /root/.cache/huggingface/datasets/cifar10/plain_text/1.0.0/951e017399fa3b30edfca56468a91ac6a03c9167c0b3f79367e777d1f9a0cf7e. Subsequent calls will reuse this data.


In [4]:
feature_extractor = ViTFeatureExtractor.from_pretrained('google/vit-base-patch16-224-in21k')
data_collator = default_data_collator

In [5]:
def preprocess_images(examples):
    images = examples['img']
    images = [np.array(image, dtype=np.uint8) for image in images]
    images = [np.moveaxis(image, source=-1, destination=0) for image in images]
    inputs = feature_extractor(images=images)
    examples['pixel_values'] = inputs['pixel_values']

    return examples

In [6]:
features = Features({
    'label': ClassLabel(
        names=['aiplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']),
    'img': Array3D(dtype='int64', shape=[3,32,32]),
    'pixel_values': Array3D(dtype='float32', shape=[3,224,224]) 
})

In [7]:
preprocessed_train_ds = train_ds.map(preprocess_images, batched=True, features=features)
preprocessed_val_ds = val_ds.map(preprocess_images, batched=True, features=features)
preprocessed_test_ds = test_ds.map(preprocess_images, batched=True, features=features) 

In [42]:
class ViTForImageClassification2(nn.Module):
    def __init__(self, num_labels=10):
        super(ViTForImageClassification2, self).__init__()
        self.vit = ViTModel.from_pretrained('google/vit-base-patch16-224-in21k')
        self.classifier = nn.Linear(self.vit.config.hidden_size, num_labels)
        self.num_labels = num_labels

    def forward(self, pixel_values, labels):
        outputs = self.vit(pixel_values=pixel_values)
        logits = self.classifier(outputs.last_hidden_state[:, 0])

        loss = None
        if labels is not None:
            loss_fct = nn.CrossEntropyLoss()
            loss = loss_fct(logits.view(-1, self.num_labels), labels.view(-1))
        
        return SequenceClassifierOutput(
            loss=loss,
            logits=logits,
            hidden_states=outputs.hidden_states,
            attentions=outputs.attentions
        )

In [43]:
args = TrainingArguments(
    f"test-cifar-10",
    evaluation_strategy='epoch',
    learning_rate=2e-5,
    per_device_train_batch_size=10,
    per_device_eval_batch_size=4,
    num_train_epochs=3,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model='accuracy',
    logging_dir='logs'
)

In [44]:
#model 
model = ViTForImageClassification2()

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return load_metric('accuracy').compute(predictions=predictions, references=labels)

In [45]:
trainer =  Trainer(
    model,
    args,
    train_dataset=preprocessed_train_ds,
    eval_dataset=preprocessed_val_ds,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

trainer.train()

outputs = trainer.predict(preprocessed_test_ds)

Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.487972,0.966000
2,1.109100,0.247846,0.968000
3,0.251700,0.204693,0.972000


In [47]:
metrics = outputs.metrics
print(metrics)

{'test_loss': 0.23655356466770172, 'test_accuracy': 0.9635, 'test_runtime': 35.4503, 'test_samples_per_second': 56.417, 'test_steps_per_second': 14.104}


In [46]:
y_pred = outputs.predictions.argmax(1)